In [1]:
from mpqp.tools import *
from mpqp import QCircuit
from mpqp.gates import *
from scipy.linalg import cossin, svd, eig
from scipy.stats import unitary_group
from mpqp.tools import pprint
import numpy as np

In [2]:
U12 = np.array([[0.517 + 0.658j, -0.131 + 0.532j, 0, 0],
                [ -0.504+0.214j, 0.731 + 0.407, 0, 0],
                [0 , 0, -0.581+0.340j, -0.644+0.364j],
                [0, 0, 0.780 - 0.208j, -0.650+0.180j]
            ])
MuxRy = np.array([[0.706, 0, -0.708, 0],
               [0, 0.927, 0, -0.374],
               [0.708, 0, 0.706, 0],
               [0, 0.374, 0, 0.927]
            ])
V12 = np.array([[0.886+0.330j, -0.325, 0, 0],
                [0.305 + 0.114j, 0.946,0, 0],
                [0, 0, -0.863 - 0.219j, 0.444 - 0.100j],
                [0, 0, 0.166 - 0.424j, -0.086+ 0.886j]
            ])
U = U12 @ MuxRy @ V12

In [ ]:
def multiplexer_decomposition(U, circuit: QCircuit, position:int, rotation: str) -> QCircuit:
    length = len(U)
    D = U[0:length//2]
    res = []
    for i in range(len(D)):
        res.append(list(D[i][length//2:length]) if rotation == "Ry" else list(D[i][0:length//2]))
    D = np.array(res)
    alphas = []
    for i in range(len(D)):
        alphas.append(-2*np.arcsin(D[i][i]) if rotation == "Ry" else 2.j*np.log(D[i][i]))

    m = build_matrix(length // 2)
    thetas = alphas @ np.linalg.inv(m)
    return gray_code_decomposition(thetas,circuit, position, rotation) 

def build_matrix(size : int):
    res = np.zeros((size,size))
    for i in range(size):
        for j in range(size):
            gray = gray_code(j)
            res[i][j] = (-1) ** (bin(i * gray).count('1'))
    return res

In [ ]:
def gray_code(n: int):
    return n ^ (n >> 1)

def unitary_SVD(U : Matrix):
    length = len(U)
    SU = U[0:(length//2)]
    SU2 = U[(length//2):length]
    g0 = []
    g1 = []
    for i in range(length//2):
        g0.append(list(SU[i][0:length//2]))
        g1.append(list(SU2[i][length//2:length]))

    
    G0 = np.array(g0)
    G1 = np.array(g1)
    
    G = G0 @ G1.conj().T

    eigvals, V = np.linalg.eig(G)
    D =  np.diag(np.sqrt(eigvals.astype(complex)))
    W = D @ V.conj().T @ G1
    D_dagg = D.conj().T
    padding = np.zeros(length//2)
    D_result = []

    for i in range(length // 2):
        D_result.append(list(D[0:length//2][i]) + list(padding))

    for i in range(length // 2):
        D_result.append(list(padding) + list(D_dagg[0:length//2][i]))

    return V,np.array(D_result),W


def gray_code_decomposition(thetas : list[float], circuit : QCircuit, position : int, rotation : str):
    for i in range(len(thetas)):

        angle = sum(-angle if bin(gray_code(i) & j).count('1') % 2 else angle for j,angle in enumerate(thetas))
        
        angle *= 2 / len(thetas)

        control_1 = gray_code(i) ^ gray_code(i+1) # CNOT's control is the changed bit of two consecutive natural numbers in gray code
        control = next(i for i in range(len(thetas)) if (control_1 >> i & 1))
        control = max(-control - 1, -circuit.nb_qubits + 1)
        if np.abs(angle) > 1e-9: # Dodge unnecessary rotations
            circuit.add(Ry(angle,position)) if rotation == "Ry" else circuit.add(Rz(angle,position))
        circuit.add(CNOT(control + position,position))
    return circuit


def _decompose(U, circuit: QCircuit, position: int =0):
    if len(U) == 2: # Decompose 1 qubit matrices
        delta = np.exp(1j * np.angle(np.linalg.det(U)) / len(U))
        V = U / delta
        beta = 2*math.acos(np.abs(V[0][0]))
        alpha = -np.angle(V[0][0]) - np.angle(V[1][0])
        gamma = -np.angle(V[0][0]) + np.angle(V[1][0])
        
        circuit.add(Rz(alpha,position))
        circuit.add(Ry(beta, position))
        circuit.add(Rz(gamma,position))
        if circuit.gphase == 0:
            circuit.gphase = delta
        else:
            circuit.gphase *= delta
        return circuit
    else: # More than 2
        length = len(U)
        (U1,U2), MuxRy, (V1,V2) = cossin(U, p=length//2, q=length//2, separate=True)
        _, debug, _ = cossin(U,p=length//2, q=length//2)
        U12 = np.zeros((len(U1)*2,len(U1)*2), dtype=np.complex128)
        for i in range(len(U1)):
            for j in range(len(U1)):
                U12[i][j] = U1[i][j]
                U12[i + len(U12) // 2][j+ len(U12) // 2] = U2[i][j]

        V12 = np.zeros((len(V1)*2,len(V1)*2), dtype=np.complex128)
        for i in range(len(V1)):
            for j in range(len(V1)):
                V12[i][j] = V1[i][j]
                V12[i + len(V12) // 2][j+ len(V12) // 2] = V2[i][j]
        
        Vu, Du, Wu = unitary_SVD(U12)
        Vv, Dv, Wv = unitary_SVD(V12)

        du = np.angle(Du.diagonal())
        for i in range(len(du) //2):
            du[i] *= -1

        dv = np.angle(Dv.diagonal())
        for i in range(len(dv) //2):
            dv[i] *= -1
        
        
        hold = _decompose(Wv, QCircuit(circuit.nb_qubits - position - 1), 0)
        print(f"Wv {position}: {check(hold.to_matrix()* hold.gphase if hold.gphase != 0 else 1, Wv)}")
        if not check(hold.to_matrix()* hold.gphase if hold.gphase != 0 else hold.to_matrix(), Wv):
            pprint(Wv)
            print()
            pprint(hold.to_matrix())
        circuit = _decompose(Wv,circuit, position+1)

        hold = gray_code_decomposition(dv, QCircuit(circuit.nb_qubits - position), 0, "Rz")
        print(f"Dv {position}: {check(hold.to_matrix()* hold.gphase  if hold.gphase != 0 else hold.to_matrix(), Dv)}")
        circuit = gray_code_decomposition(dv, circuit, position,"Rz")

        hold = _decompose(Vv, QCircuit(circuit.nb_qubits - position  - 1), 0)
        print(f"Vv {position}: {check(hold.to_matrix()* hold.gphase if hold.gphase != 0 else hold.to_matrix(), Vv)}")
        circuit = _decompose(Vv,circuit, position+1)

        hold = gray_code_decomposition(MuxRy, QCircuit(circuit.nb_qubits - position), 0, "Ry")
        print(f"Ry {position}: {check(hold.to_matrix()* hold.gphase  if hold.gphase != 0 else hold.to_matrix(), debug)}")
        circuit = gray_code_decomposition(MuxRy, circuit, position, "Ry",)

        hold = _decompose(Wu, QCircuit(circuit.nb_qubits - position - 1), 0)
        print(f"Wu {position}: {check(hold.to_matrix()* hold.gphase if hold.gphase != 0 else hold.to_matrix(), Wu)}")
        circuit = _decompose(Wu,circuit, position+1)

        hold = gray_code_decomposition(du, QCircuit(int(circuit.nb_qubits - position)), 0, "Rz")
        print(f"Du {position}: {check(hold.to_matrix()* hold.gphase if hold.gphase != 0 else hold.to_matrix(), Du)}")
        circuit = gray_code_decomposition(du, circuit, position,"Rz")

        hold = _decompose(Vu, QCircuit(circuit.nb_qubits - position - 1), 0)
        print(f"Vu {position}: {check(hold.to_matrix()* hold.gphase if hold.gphase != 0 else hold.to_matrix(), Vu)}")
        circuit = _decompose(Vu,circuit, position+1)

        return circuit


def decompose(U):
    circuit = QCircuit(int(np.log2(len(U))))
    return _decompose(U,circuit,0)


In [4]:
def check(M1,M2):
    res = True
    for i in range(len(M1)):
        for j in range(len(M2)):
            res &= np.round(M1[i][j], 5) == np.round(M2[i][j], 5)
    return res

In [ ]:
length = 8
U = unitary_group.rvs(length)
circuit = decompose(U)
#print(circuit)
print("--------------Resultat------------")
print(check(circuit.to_matrix() * circuit.gphase, U))

Wv 0: True
Dv 0: True
Vv 0: True
Ry 0: True
Wu 0: True
Du 0: True
Vu 0: True
--------------Resultat------------
True


In [266]:
for i in range(1,6):
    length = 2**i
    U = unitary_group.rvs(length)
    circuit = decompose(U)
    print(i)
    print(check(circuit.to_matrix() * circuit.gphase, U))
    print()

1
True

2
True

3
False

4
False

5
False



In [292]:
for i in range(2,8):
    length = 2**i
    U = unitary_group.rvs(length)
    position = 0
    (U1,U2), thetas, (V1,V2) = cossin(U, p=length//2, q=length//2, separate=True)
    _, Mux, _  = cossin(U, p=length//2, q=length//2)
    print(check(Mux,gray_code_decomposition(thetas,QCircuit(i),0,"Ry").to_matrix()))


True
True
True
True
True
True
